In [2]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import json 
import os
import glob
import PIL.Image as Image
import copy
import time
import shutil

from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

### Load data

#### Change project directory and run once

In [6]:
### Run once
data_path='/dtu/datasets1/02514/data_wastedetection'
project_path = os.path.join(os.getcwd(), 'Project1.2')
os.mkdir(os.path.join(project_path, 'data'))
os.mkdir(os.path.join(project_path, 'data', 'raw'))
os.mkdir(os.path.join(project_path, 'data', 'splitted'))
os.mkdir(os.path.join(project_path, 'data', 'raw', 'test'))
os.mkdir(os.path.join(project_path, 'data', 'raw', 'train'))
os.mkdir(os.path.join(project_path, 'data', 'splitted', 'test'))
os.mkdir(os.path.join(project_path, 'data', 'splitted', 'train'))

annotations = json.load(open(os.path.join('Project1.2/annotations.json')))
supercategories = {}
categories = ['Background']
for i in range(len(annotations['categories'])):
    supercategories[str(i)] = annotations['categories'][i]['supercategory']
    if annotations['categories'][i]['supercategory'] not in categories:
        categories.append(annotations['categories'][i]['supercategory'])
        os.mkdir(os.path.join(project_path, 'data', 'raw', 'test', annotations['categories'][i]['supercategory']))
        os.mkdir(os.path.join(project_path, 'data', 'raw', 'train', annotations['categories'][i]['supercategory']))
        os.mkdir(os.path.join(project_path, 'data', 'splitted', 'test', annotations['categories'][i]['supercategory']))
        os.mkdir(os.path.join(project_path, 'data', 'splitted', 'train', annotations['categories'][i]['supercategory']))


for image_id in range(len(annotations['images'])):
    if int(annotations['images'][image_id]['file_name'].split('_')[1].split('/')[0]) < 13:
        shutil.copyfile(os.path.join(data_path, annotations['images'][image_id]['file_name']), os.path.join(project_path, 'data', 'raw', 'train', supercategories[str(annotations['annotations'][image_id]['category_id'])], annotations['images'][image_id]['file_name'].split('/')[1]))
    else:
        shutil.copyfile(os.path.join(data_path, annotations['images'][image_id]['file_name']), os.path.join(project_path, 'data', 'raw', 'test', supercategories[str(annotations['annotations'][image_id]['category_id'])], annotations['images'][image_id]['file_name'].split('/')[1]))

#### Taco class

In [7]:
class Taco(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path=os.path.join(os.getcwd(), 'Project1.2', 'data', 'raw')):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path +'/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y

batch_size = 64

trainset = Taco(train=True, transform=transforms.ToTensor())
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=3)
testset = Taco(train=False, transform=transforms.ToTensor())
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)

images, labels = next(iter(train_loader))
plt.figure(figsize=(20,10))

print(f"Image shape: {images[0].numpy()[0].shape}")

for i in range(21):
    plt.subplot(5,7,i+1)
    plt.imshow(images[i].numpy()[0], 'gray')
    plt.title(labels[i].item())
    plt.axis('off')

{'Aluminium foil': 0, 'Battery': 1, 'Blister pack': 2, 'Bottle': 3, 'Bottle cap': 4, 'Broken glass': 5, 'Can': 6, 'Carton': 7, 'Cigarette': 8, 'Cup': 9, 'Food waste': 10, 'Glass jar': 11, 'Lid': 12, 'Other plastic': 13, 'Paper': 14, 'Paper bag': 15, 'Plastic bag & wrapper': 16, 'Plastic container': 17, 'Plastic glooves': 18, 'Plastic utensils': 19, 'Pop tab': 20, 'Rope & strings': 21, 'Scrap metal': 22, 'Shoe': 23, 'Squeezable tube': 24, 'Straw': 25, 'Styrofoam piece': 26, 'Unlabeled litter': 27}
{'Aluminium foil': 0, 'Battery': 1, 'Blister pack': 2, 'Bottle': 3, 'Bottle cap': 4, 'Broken glass': 5, 'Can': 6, 'Carton': 7, 'Cigarette': 8, 'Cup': 9, 'Food waste': 10, 'Glass jar': 11, 'Lid': 12, 'Other plastic': 13, 'Paper': 14, 'Paper bag': 15, 'Plastic bag & wrapper': 16, 'Plastic container': 17, 'Plastic glooves': 18, 'Plastic utensils': 19, 'Pop tab': 20, 'Rope & strings': 21, 'Scrap metal': 22, 'Shoe': 23, 'Squeezable tube': 24, 'Straw': 25, 'Styrofoam piece': 26, 'Unlabeled litter': 

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/zhome/a9/e/164550/dimos/venv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/zhome/a9/e/164550/dimos/venv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/zhome/a9/e/164550/dimos/venv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/zhome/a9/e/164550/dimos/venv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/zhome/a9/e/164550/dimos/venv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 56, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: stack 

In [2]:
# Load annotations
annotations = json.load(open(os.path.join('Project1.2', 'annotations.json')))

In [3]:
# Load images
data_path = '/dtu/datasets1/02514/data_wastedetection/'
images = {}
for image_id in range(len(annotations['images'])):
    images[image_id] = cv2.imread(os.path.join(data_path, annotations['images'][image_id]['file_name']))

In [4]:
# Get bounding box for every image 
annotations['bounding_boxes'] = {}
for image_id in range(len(annotations['images'])):
    bbxstart = 10000
    bbxfin = 0
    bbystart = 10000
    bbyfin = 0
    for id, value in enumerate(annotations['annotations'][image_id]['segmentation'][0]):
        if id % 2 == 0:
            if bbxstart > value: bbxstart = value
            if bbxfin < value: bbxfin = value
        else:
            if bbystart > value: bbystart = value
            if bbyfin < value: bbyfin = value

    annotations['bounding_boxes'][image_id] = [int(bbxstart), int(bbystart), int(bbxfin), int(bbyfin)]

### Resize images and bounding boxes

In [5]:
def resize(image, BB_start, BB_width, BB_height, size=(224, 224)):
    ### resize image
    img_resized = cv2.resize(image.copy(), size, interpolation = cv2.INTER_AREA)
    
    ### resize BB
    # get x and y ratio
    lx = size[0]/image.shape[1]
    ly = size[1]/image.shape[0]
    
    # get new (x,y), width, height
    BB_new_start = (int(BB_start[0]*lx), int(BB_start[1]*ly))
    BB_new_width = int(BB_width*ly)
    BB_new_height = int(BB_height*lx)
    
    return img_resized, BB_new_start, BB_new_width, BB_new_height

In [7]:
# # Resize images and bounding boxes
# resized_images = []
# annotations['new_bounding_boxes'] = {}
# for image_id in images:
    
#     # Resize images and bounding boxes
#     width = annotations['bounding_boxes'][image_id][2] - annotations['bounding_boxes'][image_id][0]
#     height = annotations['bounding_boxes'][image_id][3] - annotations['bounding_boxes'][image_id][1]
#     new_image, new_bb, new_width, new_height = resize(images[image_id], (annotations['bounding_boxes'][image_id][0], annotations['bounding_boxes'][image_id][1]), width, height)
    
#     # write resized images
#     resized_images.append(new_image)

#     # write new bounding boxes
#     annotations['new_bounding_boxes'][image_id] = [int(new_bb[0]), int(new_bb[1]), int(new_bb[0] + new_width), int(new_bb[1] + new_height)]